#### This notebook is designated for scraping tables to be used in EDA

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.iii.org/fact-statistic/facts-statistics-wildfires#Wildfires%20by%20year'
res = requests.get(url)
res.status_code

200

In [3]:
soup = BeautifulSoup(res.text, 'lxml')

table = soup.find('table', {'class': 'tablesorter'})
table.text

table.attrs

table_all = soup.find_all('table')
[i.text for i in table_all][1]

'Rank\nState\nEstimated number of\nproperties at risk\nRank\nState\nPercent of\nproperties at risk\n1\nCalifornia\n2,040,600\n1\nMontana\n29%\n2\nTexas\n717,800\n2\nIdaho\n26\n3\nColorado\n373,900\n3\nColorado\n17\n4\nArizona\n242,200\n4\nCalifornia\n15\n5\nIdaho\n175,000\n5\nNew Mexico\n15\n6\nWashington\n155,500\n6\nUtah\n14\n7\nOklahoma\n153,400\n7\nWyoming\n14\n8\nOregon\n147,500\n8\nArizona\n9\n9\nMontana\n137,800\n9\nOklahoma\n9\n10\nUtah\n136,000\n10\nOregon\n9\n11\nNew Mexico\n131,600\n11\nTexas\n7\n12\nNevada\n67,100\n12\nNevada\n6\n13\nWyoming\n36,800\n13\nWashington\n5\n'

### Wildfire Risk by Number of Properties at Risk (2021)

In [4]:
states = []
for row in table.find_all('tr')[1:]:
    state = {}
    state['rank'] = row.find_all('td')[0].text.strip()
    state['state'] = row.find_all('td')[1].text.strip()
    state['est_num_of_props_at_risk'] = row.find_all('td')[2].text.strip()
    states.append(state)
high_wildfire_risk_df = pd.DataFrame(states)
high_wildfire_risk_df

,rank,state,est_num_of_props_at_risk
0,1,California,"2,040,600"
1,2,Texas,"717,800"
2,3,Colorado,"373,900"
3,4,Arizona,"242,200"
4,5,Idaho,"175,000"
5,6,Washington,"155,500"
6,7,Oklahoma,"153,400"
7,8,Oregon,"147,500"
8,9,Montana,"137,800"
9,10,Utah,"136,000"


In [5]:
def remove_punctuation(column, punc):
    return column.replace(punc, '', regex=True, inplace=True)

remove_punctuation(high_wildfire_risk_df['est_num_of_props_at_risk'], ',')

In [6]:
# Set columns to integer
high_wildfire_risk_df['est_num_of_props_at_risk'] = high_wildfire_risk_df['est_num_of_props_at_risk'].astype(int)
high_wildfire_risk_df['rank'] = high_wildfire_risk_df['rank'].astype(int)

In [7]:
# Sort df
high_wildfire_risk_df.sort_values('est_num_of_props_at_risk', ascending=False, inplace=True)
high_wildfire_risk_df.head()

,rank,state,est_num_of_props_at_risk
0,1,California,2040600
1,2,Texas,717800
2,3,Colorado,373900
3,4,Arizona,242200
4,5,Idaho,175000


In [9]:
high_wildfire_risk_df.to_csv('./Data/wildfire_risk_by_number_of_properties_2021.csv', index=False)

### Wildfire Risk by Percent (2021)

In [10]:
states = []
for row in table.find_all('tr')[1:]:
    state = {}
    state['rank'] = row.find_all('td')[3].text.strip()
    state['state'] = row.find_all('td')[4].text.strip()
    state['perc_props_at_risk'] = row.find_all('td')[5].text.strip()
    states.append(state)
high_wildfire_risk_perc_df = pd.DataFrame(states)
high_wildfire_risk_perc_df.head()

,rank,state,perc_props_at_risk
0,1,Montana,29%
1,2,Idaho,26
2,3,Colorado,17
3,4,California,15
4,5,New Mexico,15


In [11]:
remove_punctuation(high_wildfire_risk_perc_df['perc_props_at_risk'], '%')

In [12]:
# Set columns to integer
high_wildfire_risk_perc_df['perc_props_at_risk'] = high_wildfire_risk_perc_df['perc_props_at_risk'].astype(float)
high_wildfire_risk_perc_df['rank'] = high_wildfire_risk_perc_df['rank'].astype(int)

In [13]:
# Sort df
high_wildfire_risk_perc_df.sort_values('perc_props_at_risk', ascending=False)
high_wildfire_risk_perc_df.head()

,rank,state,perc_props_at_risk
0,1,Montana,29.0
1,2,Idaho,26.0
2,3,Colorado,17.0
3,4,California,15.0
4,5,New Mexico,15.0


In [14]:
high_wildfire_risk_perc_df.to_csv('./Data/wildfire_risk_by_percent_2021.csv', index=False)

### Wildfires by State (2021)

In [15]:
second_soup = BeautifulSoup(res.text, 'lxml')

second_table = second_soup.find_all('table', {'class': 'tablesorter'})[1]
second_table.text[:50]

'State\nNumber of fires\nNumber of acres burned\xa0\nAlab'

In [16]:
states = []
for row in second_table.find_all('tr')[1:]:
    state = {}
    state['state'] = row.find_all('td')[0].text.strip()
    state['num_fires'] = row.find_all('td')[1].text.strip()
    state['num_acres_burned'] = row.find_all('td')[2].text.strip()
    states.append(state)
wildfires_by_state_df = pd.DataFrame(states)
wildfires_by_state_df.head()

,state,num_fires,num_acres_burned
0,Alabama,"1,040","22,055"
1,Alaska,384,"253,357"
2,Arizona,"1,773","524,428"
3,Arkansas,378,"17,003"
4,California,"9,260","2,233,666"


In [17]:
remove_punctuation(wildfires_by_state_df['num_fires'], ',')

In [18]:
remove_punctuation(wildfires_by_state_df['num_acres_burned'], ',')

In [19]:
# Set columns to integer
wildfires_by_state_df['num_fires'] = wildfires_by_state_df['num_fires'].astype(int)
wildfires_by_state_df['num_acres_burned'] = wildfires_by_state_df['num_acres_burned'].astype(int)

In [20]:
# sort df
wildfires_by_state_df.sort_values('state', ascending=True)
wildfires_by_state_df.head()

,state,num_fires,num_acres_burned
0,Alabama,1040,22055
1,Alaska,384,253357
2,Arizona,1773,524428
3,Arkansas,378,17003
4,California,9260,2233666


In [21]:
wildfires_by_state_df.to_csv('./Data/wildfires_by_state_2021.csv', index=False)

### Top 10 States for Wildfires Ranked by Number of Fires(2021)

In [22]:
third_soup = BeautifulSoup(res.text, 'lxml')

third_table = third_soup.find_all('table', {'class': 'tablesorter'})[2]
third_table.text[:50]

'Rank\nState\nNumber of fires\nRank\nState\nNumber of ac'

In [23]:
states = []
for row in third_table.find_all('tr')[1:]:
    state = {}
    state['rank'] = row.find_all('td')[0].text.strip()
    state['state'] = row.find_all('td')[1].text.strip()
    state['num_fires'] = row.find_all('td')[2].text.strip()
    states.append(state)
top_10_no_of_wildfires = pd.DataFrame(states)
top_10_no_of_wildfires.head()

,rank,state,num_fires
0,1,California,"9,260"
1,2,Texas,"5,576"
2,3,North Carolina,"5,151"
3,4,Montana,"2,573"
4,5,Florida,"2,262"


In [24]:
remove_punctuation(top_10_no_of_wildfires['num_fires'], ',')

In [25]:
# Convert series to integer
top_10_no_of_wildfires['num_fires'] = top_10_no_of_wildfires['num_fires'].astype(int)

In [26]:
# sort df
top_10_no_of_wildfires.sort_values('num_fires', ascending=False)
top_10_no_of_wildfires.head()

,rank,state,num_fires
0,1,California,9260
1,2,Texas,5576
2,3,North Carolina,5151
3,4,Montana,2573
4,5,Florida,2262


In [28]:
top_10_no_of_wildfires.to_csv('./Data/top_10_states_for_wf_2021.csv', index=False)

### Top 10 States for Wildfires Ranked by Number of Acres Burned (2021)

In [29]:
fourth_soup = BeautifulSoup(res.text, 'lxml')

fourth_table = fourth_soup.find_all('table', {'class': 'tablesorter'})[2]
fourth_table.text[:50]

'Rank\nState\nNumber of fires\nRank\nState\nNumber of ac'

In [30]:
states = []
for row in fourth_table.find_all('tr')[1:]:
    state = {}
    state['rank'] = row.find_all('td')[3].text.strip()
    state['state'] = row.find_all('td')[4].text.strip()
    state['num_acres_burned'] = row.find_all('td')[5].text.strip()
    states.append(state)
top_10_acres_wildfires = pd.DataFrame(states)
top_10_acres_wildfires.head()

,rank,state,num_acres_burned
0,1,California,"2,233,666"
1,2,Oregon,"828,777"
2,3,Montana,"747,678"
3,4,Washisngton,"674,222"
4,5,Arizona,"524,428"


In [31]:
remove_punctuation(top_10_acres_wildfires['num_acres_burned'], ',')

In [32]:
# Convert series to int
top_10_acres_wildfires['num_acres_burned'] = top_10_acres_wildfires['num_acres_burned'].astype(int)

In [33]:
# sort df
top_10_acres_wildfires.sort_values('num_acres_burned', ascending=False)
top_10_acres_wildfires.head()

,rank,state,num_acres_burned
0,1,California,2233666
1,2,Oregon,828777
2,3,Montana,747678
3,4,Washisngton,674222
4,5,Arizona,524428


In [34]:
top_10_acres_wildfires.to_csv('./Data/top_10_states_for_wf_acres_2021.csv', index=False)

### Top 10 Costliest Wildfires in the United States (in millions)

In [35]:
fifth_soup = BeautifulSoup(res.text, 'lxml')

fifth_table = fifth_soup.find_all('table', {'class': 'tablesorter'})[3]
fifth_table.text[:50]

'\xa0\n\xa0\n\xa0\nEstimated insured loss\nRank\nYear\nName\nDollar'

In [36]:
fires = []
for row in fifth_table.find_all('tr')[2:]:
    fire = {}
    fire['rank'] = row.find_all('td')[0].text.strip()
    fire['year'] = row.find_all('td')[1].text.strip()
    fire['name'] = row.find_all('td')[2].text.strip()
    fire['dollars_when_occured'] = row.find_all('td')[3].text.strip()
    fire['in_2021_dollars'] = row.find_all('td')[4].text.strip()
    fires.append(fire)
costliest_fires_df = pd.DataFrame(fires)
costliest_fires_df.head()

,rank,year,name,dollars_when_occured,in_2021_dollars
0,1,2018,Camp Fire,"$10,000","$10,750"
1,2,2017,Tubbs Fire,"8,700","9,560"
2,3,2018,Woolsey Fire,"4,200","4,520"
3,4,1991,Oakland Fire (Tunnel),"1,700","3,350"
4,5,2017,Atlas Fire,"3,000","3,300"


In [37]:
remove_punctuation(costliest_fires_df['dollars_when_occured'], ',')

In [38]:
remove_punctuation(costliest_fires_df['in_2021_dollars'], ',')

In [39]:
# remove $ sign (for some reason, function didn't pick it up)
costliest_fires_df['dollars_when_occured'].replace('$10000','10000', inplace=True)
costliest_fires_df['in_2021_dollars'].replace('$10750','10750', inplace=True)

In [40]:
# Convert series to int
costliest_fires_df['dollars_when_occured'] = costliest_fires_df['dollars_when_occured'].astype(int)
costliest_fires_df['in_2021_dollars'] = costliest_fires_df['in_2021_dollars'].astype(int)

In [41]:
# sort df
costliest_fires_df.sort_values('in_2021_dollars', ascending=False)
costliest_fires_df.head()

,rank,year,name,dollars_when_occured,in_2021_dollars
0,1,2018,Camp Fire,10000,10750
1,2,2017,Tubbs Fire,8700,9560
2,3,2018,Woolsey Fire,4200,4520
3,4,1991,Oakland Fire (Tunnel),1700,3350
4,5,2017,Atlas Fire,3000,3300


In [42]:
costliest_fires_df.to_csv('./Data/top_10_costliest_wf.csv', index=False)

### Top 10 Largest California Wildfires

In [43]:
sixth_soup = BeautifulSoup(res.text, 'lxml')

sixth_table = sixth_soup.find_all('table', {'class': 'tablesorter'})[4]
sixth_table.text[:50]

'Rank\nFire name (cause)\nDate\nCounty\nAcres\nStructure'

In [44]:
fires_ca = []
for row in sixth_table.find_all('tr')[1:]:
    fire = {}
    fire['rank'] = row.find_all('td')[0].text.strip()
    fire['fire_name_cause'] = row.find_all('td')[1].text.strip()
    fire['date'] = row.find_all('td')[2].text.strip()
    fire['county'] = row.find_all('td')[3].text.strip()
    fires_ca.append(fire)
ca_largest_wf_df = pd.DataFrame(fires_ca)
ca_largest_wf_df.head()

,rank,fire_name_cause,date,county
0,1,August Complex (Lightning),August 2020,"Mendocino, Humboldt, Trinity, Tehama, Glenn, L..."
1,2,Dixie (Under investigation),July 2021,"Butte, Plumas, Lassen, Shasta, and Tehama"
2,3,Mendocino Complex (Human related),July 2018,"Colusa, Lake,Mendocino and Glenn"
3,4,SCU Lightning Complex (Lightning),August 2020,"Stanislaus, Santa Clara, Alameda, Contra Costa..."
4,5,Creek Fire (Undetermined),September 2020,Fresno and Madera


In [45]:
ca_largest_wf_df.to_csv('./Data/top_10_largest_ca_wf.csv', index=False)

### Top 10 Most Destructive California Wildfires

In [46]:
seventh_soup = BeautifulSoup(res.text, 'lxml')

seventh_table = seventh_soup.find_all('table', {'class': 'tablesorter'})[5]
seventh_table.text[:50]

'Rank\nFire name and cause\nDate\nCounty\nAcres\nStructu'

In [47]:
destructive_fires = []
for row in seventh_table.find_all('tr')[1:]:
    fire = {}
    fire['rank'] = row.find_all('td')[0].text.strip()
    fire['fire_name_cause'] = row.find_all('td')[1].text.strip()
    fire['date'] = row.find_all('td')[2].text.strip()
    fire['county'] = row.find_all('td')[3].text.strip()
    fire['acres'] = row.find_all('td')[4].text.strip()
    fire['structures'] = row.find_all('td')[5].text.strip()
    destructive_fires.append(fire)
ca_destructive_wf_df = pd.DataFrame(destructive_fires)
ca_destructive_wf_df.head()

,rank,fire_name_cause,date,county,acres,structures
0,1,Camp Fire (Power lines),November 2018,Butte,"153,336","18,804"
1,2,Tubbs (Electrical),October 2017,Napa and Sonoma,"36,807","5,636"
2,3,Tunnel - Oakland Hills (Rekindle),October 1991,Alameda,"1,600","2,900"
3,4,Cedar (Human related),October 2003,San Diego,"273,246","2,820"
4,5,North Complex (Lightning),August 2020,"Butte, Plumas and Yuba","318,935","2,352"


In [48]:
remove_punctuation(ca_destructive_wf_df['acres'], ',')

In [49]:
remove_punctuation(ca_destructive_wf_df['structures'], ',')

In [50]:
# Convert series to int
ca_destructive_wf_df['acres'] = ca_destructive_wf_df['acres'].astype(int)
ca_destructive_wf_df['structures'] = ca_destructive_wf_df['structures'].astype(int)

In [51]:
ca_destructive_wf_df.to_csv('./Data/top_10_destructive_ca_wf.csv', index=False)

### Top 10 Deadliest California Wildfires

In [52]:
eighth_soup = BeautifulSoup(res.text, 'lxml')

eighth_table = eighth_soup.find_all('table', {'class': 'tablesorter'})[6]
eighth_table.text[:50]

'Rank\nFire name and cause\nDate\nCounty\nAcres\nStructu'

In [53]:
deadliest_fires = []
for row in seventh_table.find_all('tr')[1:]:
    fire = {}
    fire['rank'] = row.find_all('td')[0].text.strip()
    fire['fire_name_cause'] = row.find_all('td')[1].text.strip()
    fire['date'] = row.find_all('td')[2].text.strip()
    fire['county'] = row.find_all('td')[3].text.strip()
    fire['acres'] = row.find_all('td')[4].text.strip()
    fire['structures'] = row.find_all('td')[5].text.strip()
    fire['deaths'] = row.find_all('td')[6].text.strip()
    deadliest_fires.append(fire)
ca_deadliest_wf_df = pd.DataFrame(deadliest_fires)
ca_deadliest_wf_df.head()

,rank,fire_name_cause,date,county,acres,structures,deaths
0,1,Camp Fire (Power lines),November 2018,Butte,"153,336","18,804",85
1,2,Tubbs (Electrical),October 2017,Napa and Sonoma,"36,807","5,636",22
2,3,Tunnel - Oakland Hills (Rekindle),October 1991,Alameda,"1,600","2,900",25
3,4,Cedar (Human related),October 2003,San Diego,"273,246","2,820",15
4,5,North Complex (Lightning),August 2020,"Butte, Plumas and Yuba","318,935","2,352",15


In [54]:
remove_punctuation(ca_deadliest_wf_df['acres'], ',')

In [55]:
remove_punctuation(ca_deadliest_wf_df['structures'], ',')

In [56]:
# Convert series to int
ca_deadliest_wf_df['acres'] = ca_deadliest_wf_df['acres'].astype(int)
ca_deadliest_wf_df['structures'] = ca_deadliest_wf_df['structures'].astype(int)
ca_deadliest_wf_df['deaths'] = ca_deadliest_wf_df['deaths'].astype(int)

In [57]:
ca_deadliest_wf_df.to_csv('./Data/top_10_deadliest_ca_wf.csv', index=False)